# Task Distillation

## Metrics

In [12]:
import os
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the base folders
model_types = [
    "plumber_img_prompt", "plumber_img_text_proj_img_prompt_cls_LP",
    "plumber_img_proj", "plumber_img_prompt_cls_LP", 
    "plumber_text_proj_img_prompt", "plumber_img_proj_cls_LP", 
    "plumber_img_text_proj", "plumber_text_proj_img_prompt_cls_LP", 
    "plumber_img_proj_img_prompt", "plumber_img_text_proj_cls_LP", 
    "plumber_img_proj_img_prompt_cls_LP", "plumber_img_text_proj_img_prompt"
]
corruption_folders = [
    "brightness", "contrast", "defocus_blur", "elastic_transform", "fog",
    "frost", "gaussian_blur", "gaussian_noise", "glass_blur", "impulse_noise",
    "jpeg_compression", "motion_blur", "pixelate", "saturate", "shot_noise",
    "snow", "spatter", "speckle_noise", "zoom_blur"
]
corruption_folders = [f"cifar10-c_{folder}_4" for folder in corruption_folders]

# corruption_folders = ["clipart", "infograph", "painting", "quickdraw", "real", "sketch"]
# corruption_folders = [f"{folder}" for folder in corruption_folders]


# corruption_folders = [""]

dataset = "cifar100"
save_data_name = "cifar100"
model_name = "resnet50"

epoch = 199

# Directory where these folders are located
base_directory = f"logs/{dataset}/{model_name}"  # Replace with the actual path

remaining_dir = f"_clsEpoch_{epoch}_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/logs"

def extract_metrics_from_json(file_path, model_type):

    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

data_list = []
# Iterate over the base folders and corruption folders
for model_type in model_types:
    path = os.path.join(base_directory, model_type)
    for corruption_folder in corruption_folders:
        metrics_file = os.path.join(path, remaining_dir, "test_task_distillation.json")
        if os.path.exists(metrics_file):
            metrics_data = extract_metrics_from_json(metrics_file, model_type)
            metrics_data["Model Type"] = model_type
            print(metrics_data)
            data_list.append(metrics_data)
        else:
            print("Metrics file not found for", metrics_file)

Metrics file not found for logs/cifar100/resnet50/plumber_img_prompt/_clsEpoch_199_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/logs/test_task_distillation.json
Metrics file not found for logs/cifar100/resnet50/plumber_img_prompt/_clsEpoch_199_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/logs/test_task_distillation.json
Metrics file not found for logs/cifar100/resnet50/plumber_img_prompt/_clsEpoch_199_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/logs/test_task_distillation.json
Metrics file not found for logs/cifar100/resnet50/plumber_img_prompt/_clsEpoch_199_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/logs/test_task_distillation.json
Metrics file not found for logs/cifar100/resnet50/plumber_img_prompt/_clsEpoch_199_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/logs/test_task_distillation.json
Metrics file not found for logs/cif

In [8]:
# Display the dataframe
df = pd.DataFrame(data_list)

new_row_values = {
    'acc': df['base_model_acc'][0],
    'ece': df['base_model_ece'][0],
    'precision': df['base_model_precision'][0],
    'recall': df['base_model_recall'][0],
    'Model Type': 'Task Model'
}

# Create a new DataFrame for the new row
new_row_df = pd.DataFrame([new_row_values])

# Drop the 'base_model_' columns
df = df.drop(columns=['base_model_acc', 'base_model_ece', 'base_model_precision', 'base_model_recall'])

# Rename 'plumber_' columns
df.columns = [col.replace('plumber_', '') for col in df.columns]

# Add the new row to the top of the DataFrame
df = pd.concat([new_row_df, df], ignore_index=True)


# Save the dataframe
# df.to_csv(f"Distill:{save_data_name}-{model_name}.csv", index=False)


# Group by 'Model Type' and 'Feature Type' and calculate mean for specific columns
grouped_df = df.groupby(['Model Type']).agg({
    'acc': 'mean',
    'ece': 'mean',
    'precision': 'mean',
    'recall': 'mean',
}).reset_index()

model_mapping = {
    "Task Model": "Task Model",
    "Image (Prompt)": "plumber_img_prompt",
    "Image (Project)": "plumber_img_proj",
    "Image (Prompt + Project)": "plumber_img_proj_img_prompt",
    "Image (Prompt) + Text (Cls Prompt)": "plumber_img_prompt_cls_LP",
    "Image (Prompt) + Text (Project)": "plumber_text_proj_img_prompt",
    "Image (Prompt) + Text (Prompt + Project)": "plumber_text_proj_img_prompt_cls_LP",
    "Image (Project) + Text (Cls Prompt)": "plumber_img_proj_cls_LP",
    "Image (Project) + Text (Project)": "plumber_img_text_proj",
    "Image (Project) + Text (Prompt + Project)": "plumber_img_text_proj_cls_LP",
    "Image (Prompt + Project) + Text (Cls Prompt)": "plumber_img_proj_img_prompt_cls_LP",
    "Image (Prompt + Project) + Text (Project)": "plumber_img_text_proj_img_prompt",
    "Image (Prompt + Project) + Text (Prompt + Project)": "plumber_img_text_proj_img_prompt_cls_LP"
}
# Reverse the dictionary for replacement (value to key)
reverse_mapping = {v: k for k, v in model_mapping.items()}

# Replace the 'Model Type' in the DataFrame using the reversed mapping
grouped_df['Model Type'] = grouped_df['Model Type'].map(reverse_mapping)

# Create a custom sort order based on the order of keys in model_mapping
sort_order = {k: i for i, k in enumerate(model_mapping.keys())}

# Add a temporary sorting column based on the custom order
grouped_df['Sort Order'] = grouped_df['Model Type'].map(sort_order)

# Sort the DataFrame by this custom order and drop the temporary column
grouped_df = grouped_df.sort_values(by='Sort Order').drop('Sort Order', axis=1)

# Display the sorted DataFrame
print(grouped_df)
grouped_df.to_csv(f"Distill:{save_data_name}-{model_name}-grouped.csv", index=False)

                                           Model Type       acc       ece  \
0                                          Task Model  0.682951  0.192366   
5                                      Image (Prompt)  0.764438  0.093513   
1                                     Image (Project)  0.871835  0.268414   
3                            Image (Prompt + Project)  0.464893  0.031753   
6                  Image (Prompt) + Text (Cls Prompt)  0.748616  0.085622   
11                    Image (Prompt) + Text (Project)  0.350079  0.037044   
12           Image (Prompt) + Text (Prompt + Project)  0.436412  0.034039   
2                 Image (Project) + Text (Cls Prompt)  0.872528  0.275495   
7                    Image (Project) + Text (Project)  0.713113  0.365735   
8           Image (Project) + Text (Prompt + Project)  0.788271  0.322210   
4        Image (Prompt + Project) + Text (Cls Prompt)  0.405063  0.073370   
9           Image (Prompt + Project) + Text (Project)  0.580795  0.086238   

In [9]:
def extract_metrics_from_json(file_path, model_type):
    corruption_data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the JSON string to a dictionary
            data = json.loads(line.strip())
            data["Model Type"] = model_type
            corruption_data.append(data)

    return corruption_data

data_list = []
# Iterate over the base folders and corruption folders
for model_type in model_types:
    path = os.path.join(base_directory, model_type)
    metrics_file = os.path.join(path, remaining_dir, "test_task_distillation_corruption.json")
    if os.path.exists(metrics_file):
        metrics_data = extract_metrics_from_json(metrics_file, model_type)
        data_list.extend(metrics_data)
    else:
        print("Metrics file not found for", metrics_file)

In [10]:
# Display the dataframe
df = pd.DataFrame(data_list)
# Get new row values for each of the corruption types in the dataframe
available_corruptions = df['corruption'].unique()
new_row_values = {}

# I want this row to have model type, corruption type, acc, ece, precision and recall values from the base model
for corruption in available_corruptions:
    # Get the row for the base model and the current corruption type
    base_model_row = df[df['corruption'] == corruption]
    # Get the accuracy, ece, precision and recall values
    acc = base_model_row['base_model_acc'].values[0]
    ece = base_model_row['base_model_ece'].values[0]
    precision = base_model_row['base_model_precision'].values[0]
    recall = base_model_row['base_model_recall'].values[0]
    severity = base_model_row['severity'].values[0]
    # Add the values to the dictionary
    new_row_values[corruption] = {
        'Model Type': 'Task Model',
        'corruption': corruption,
        'severity': severity,
        'acc': acc,
        'ece': ece,
        'precision': precision,
        'recall': recall
    }

# Create a new DataFrame for the new row
new_row_df = pd.DataFrame(new_row_values).T
# print(new_row_df)
# Drop the 'base_model_' columns
df = df.drop(columns=['base_model_acc', 'base_model_ece', 'base_model_precision', 'base_model_recall'])

# Rename 'plumber_' columns
df.columns = [col.replace('plumber_', '') for col in df.columns]

# Add the new row to the top of the DataFrame
df = pd.concat([new_row_df, df], ignore_index=True)

# Save the dataframe
# df.to_csv(f"Distill:{save_data_name}-{model_name}.csv", index=False)

# Group by 'Model Type' and 'Feature Type' and calculate mean for specific columns
grouped_df = df.groupby(['Model Type']).agg({
    'acc': 'mean',
    'ece': 'mean',
    'precision': 'mean',
    'recall': 'mean',
}).reset_index()

model_mapping = {
    "Task Model": "Task Model",
    "Image (Prompt)": "plumber_img_prompt",
    "Image (Project)": "plumber_img_proj",
    "Image (Prompt + Project)": "plumber_img_proj_img_prompt",
    "Image (Prompt) + Text (Cls Prompt)": "plumber_img_prompt_cls_LP",
    "Image (Prompt) + Text (Project)": "plumber_text_proj_img_prompt",
    "Image (Prompt) + Text (Prompt + Project)": "plumber_text_proj_img_prompt_cls_LP",
    "Image (Project) + Text (Cls Prompt)": "plumber_img_proj_cls_LP",
    "Image (Project) + Text (Project)": "plumber_img_text_proj",
    "Image (Project) + Text (Prompt + Project)": "plumber_img_text_proj_cls_LP",
    "Image (Prompt + Project) + Text (Cls Prompt)": "plumber_img_proj_img_prompt_cls_LP",
    "Image (Prompt + Project) + Text (Project)": "plumber_img_text_proj_img_prompt",
    "Image (Prompt + Project) + Text (Prompt + Project)": "plumber_img_text_proj_img_prompt_cls_LP"
}
# Reverse the dictionary for replacement (value to key)
reverse_mapping = {v: k for k, v in model_mapping.items()}

# Replace the 'Model Type' in the DataFrame using the reversed mapping
grouped_df['Model Type'] = grouped_df['Model Type'].map(reverse_mapping)

# Create a custom sort order based on the order of keys in model_mapping
sort_order = {k: i for i, k in enumerate(model_mapping.keys())}

# Add a temporary sorting column based on the custom order
grouped_df['Sort Order'] = grouped_df['Model Type'].map(sort_order)

# Sort the DataFrame by this custom order and drop the temporary column
grouped_df = grouped_df.sort_values(by='Sort Order').drop('Sort Order', axis=1)

# Display the sorted DataFrame
print(grouped_df)
grouped_df.to_csv(f"Distill:corr-{save_data_name}-{model_name}-grouped.csv", index=False)

                                           Model Type       acc       ece  \
0                                          Task Model  0.577677  0.250648   
5                                      Image (Prompt)  0.563952  0.088213   
1                                     Image (Project)  0.615272  0.195565   
3                            Image (Prompt + Project)  0.328578  0.091283   
6                  Image (Prompt) + Text (Cls Prompt)  0.512081  0.109827   
11                    Image (Prompt) + Text (Project)  0.252571  0.090889   
12           Image (Prompt) + Text (Prompt + Project)  0.299935  0.121622   
2                 Image (Project) + Text (Cls Prompt)  0.622481  0.193415   
7                    Image (Project) + Text (Project)  0.447431  0.195982   
8           Image (Project) + Text (Prompt + Project)  0.540556  0.206195   
4        Image (Prompt + Project) + Text (Cls Prompt)  0.290332  0.146399   
9           Image (Prompt + Project) + Text (Project)  0.350735  0.176972   

# Failure Detection

## V1

In [ ]:
import os
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the base folders
model_types = [
    "plumber_img_prompt", "plumber_img_text_proj_img_prompt_cls_LP",
    "plumber_img_proj", "plumber_img_prompt_cls_LP", 
    "plumber_text_proj_img_prompt", "plumber_img_proj_cls_LP", 
    "plumber_img_text_proj", "plumber_text_proj_img_prompt_cls_LP", 
    "plumber_img_proj_img_prompt", "plumber_img_text_proj_cls_LP", 
    "plumber_img_proj_img_prompt_cls_LP", "plumber_img_text_proj_img_prompt"
]
corruption_folders = [
    "brightness", "contrast", "defocus_blur", "elastic_transform", "fog",
    "frost", "gaussian_blur", "gaussian_noise", "glass_blur", "impulse_noise",
    "jpeg_compression", "motion_blur", "pixelate", "saturate", "shot_noise",
    "snow", "spatter", "speckle_noise", "zoom_blur"
]
corruption_folders = ["clipart", "infograph", "painting", "quickdraw", "real", "sketch"]
# corruption_folders = [f"domainnet_{folder}_" for folder in corruption_folders]

# corruption_folders = [f"cifar10-c_{folder}_4" for folder in corruption_folders]

corruption_folders = [""]

dataset = "cifar10"
save_data_name = "cifar10"
model_name = "SimpleCNN"

epoch = 29

# Directory where these folders are located
base_directory = f"logs/{dataset}/{model_name}"  # Replace with the actual path

remaining_dir = f"_clsEpoch_{epoch}_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/failure_detector"

# Function to extract metrics from a file
def extract_metrics(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        data = {}
        for line in lines:
            if line.startswith("proj_features") or line.startswith("classifier_features") or line.startswith("clip_features"):
                parts = line.split()
                data_type = parts[0]
                metrics = {
                    "Accuracy": float(parts[2].strip('%,')),
                    "Precision": float(parts[4].strip('%,')),
                    "Recall": float(parts[6].strip('%,')),
                    "F1 Score": float(parts[9].strip('%'))
                }
                data[data_type] = metrics
        return data

def extract_metrics_from_json(file_path):
    # classifier_features_metrics_single_svm
    model_type = file_path.split("/")[-1].split("_metrics")[0]

    # if model_type == "plumber_img_prompt" add update the model file_path to also include proj_features_metrics_single_svm and clip_features_metrics_single_svm
    if model_type == "plumber_img_prompt":
        file_paths = [file_path] + [file_path.replace("proj_features", "classifier_features")] + [file_path.replace("proj_features", "clip_features")]
    else:
        file_paths = [file_path]
    
    all_metrics = {}
    for file_path in file_paths:
        metrics = {}
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        metrics['task_model_acc'] = accuracy_score(data["gt_labels"], data["task_pred"])
        metrics['estimated_acc'] = sum(data["correct_svm_pred"]) / len(data["correct_svm_pred"])
        metrics['estimation_gap'] = data["estimation_gap"]
        metrics['failure_svm_accuracy'] = data["accu_failure_pred"]
        metrics['sucess_svm_accuracy'] = data["accu_success_pred"]

        all_metrics[model_type] = metrics

    return all_metrics

In [ ]:

data_list = []
# Iterate over the base folders and corruption folders
for model_type in model_types:
    path = os.path.join(base_directory, model_type)
    for corruption_folder in corruption_folders:
        corruption_path = os.path.join(path, remaining_dir, f"{corruption_folder}")
        metrics_file = os.path.join(corruption_path, "single_svm_metrics.txt")
        if os.path.exists(metrics_file):
            metrics_data = extract_metrics(metrics_file)
            print(metrics_data)
            for key, values in metrics_data.items():
                row = {"Model Type": model_type, 
                                "Corruption": corruption_folder,
                                "Feature Type": key,
                                **values}
                
                data_list.append(row)
        else:
            print("Metrics file not found for", metrics_file)



In [ ]:
# Display the dataframe
df = pd.DataFrame(data_list)

# Save the dataframe
df.to_csv(f"{save_data_name}-{model_name}.csv", index=False)
print(df)

In [ ]:
# Create different dataframes for different corruption types
df_corruptions = {}
for corruption_folder in corruption_folders:
    df_corruption = df[df["Corruption"] == corruption_folder]
    df_corruptions[corruption_folder] = df_corruption
    print(df_corruption)

In [ ]:
# Group by 'Model Type' and 'Feature Type' and calculate mean for specific columns
grouped_df = df.groupby(['Model Type', 'Feature Type']).agg({
    'Accuracy': 'mean',
    'Precision': 'mean',
    'Recall': 'mean',
    'F1 Score': 'mean'
}).reset_index()

# Display the new dataframe with mean values
print(grouped_df)
grouped_df.to_csv(f"{save_data_name}-{model_name}-grouped.csv", index=False)

In [ ]:
model_mapping = {
    "Image (Prompt)": "plumber_img_prompt",
    "Image (Project)": "plumber_img_proj",
    "Image (Prompt + Project)": "plumber_img_proj_img_prompt",
    "Image (Prompt) + Text (Cls Prompt)": "plumber_img_prompt_cls_LP",
    "Image (Prompt) + Text (Project)": "plumber_text_proj_img_prompt",
    "Image (Prompt) + Text (Prompt + Project)": "plumber_text_proj_img_prompt_cls_LP",
    "Image (Project) + Text (Cls Prompt)": "plumber_img_proj_cls_LP",
    "Image (Project) + Text (Project)": "plumber_img_text_proj",
    "Image (Project) + Text (Prompt + Project)": "plumber_img_text_proj_cls_LP",
    "Image (Prompt + Project) + Text (Cls Prompt)": "plumber_img_proj_img_prompt_cls_LP",
    "Image (Prompt + Project) + Text (Project)": "plumber_img_text_proj_img_prompt",
    "Image (Prompt + Project) + Text (Prompt + Project)": "plumber_img_text_proj_img_prompt_cls_LP"
}
# Reverse the dictionary for replacement (value to key)
reverse_mapping = {v: k for k, v in model_mapping.items()}

# Replace the 'Model Type' in the DataFrame using the reversed mapping
grouped_df['Model Type'] = grouped_df['Model Type'].map(reverse_mapping)

# Create a custom sort order based on the order of keys in model_mapping
sort_order = {k: i for i, k in enumerate(model_mapping.keys())}

# Add a temporary sorting column based on the custom order
grouped_df['Sort Order'] = grouped_df['Model Type'].map(sort_order)

# Sort the DataFrame by this custom order and drop the temporary column
grouped_df = grouped_df.sort_values(by='Sort Order').drop('Sort Order', axis=1)

# Display the sorted DataFrame
print(grouped_df)
# grouped_df.to_csv("Waterbirds-resnet18-grouped-sorted.csv", index=False)

In [ ]:
# list(grouped_df["Accuracy"])
# list(grouped_df["Precision"])
list(grouped_df["Recall"])

In [ ]:
# Group each corruption type by 'Model Type' and 'Feature Type' and calculate mean for specific columns and remap the 'Model Type' using the reversed mapping
# Also sort the DataFrame by the custom order

# Create a custom sort order based on the order of keys in model_mapping
sort_order = {k: i for i, k in enumerate(model_mapping.keys())}

grouped_corruptions = {}
for corruption_folder in corruption_folders:
    df_corruption = df_corruptions[corruption_folder]
    grouped_df_corruption = df_corruption.groupby(['Model Type', 'Feature Type']).agg({
        'Accuracy': 'mean',
        'Precision': 'mean',
        'Recall': 'mean',
        'F1 Score': 'mean'
    }).reset_index()
    grouped_df_corruption['Model Type'] = grouped_df_corruption['Model Type'].map(reverse_mapping)
    grouped_df_corruption['Sort Order'] = grouped_df_corruption['Model Type'].map(sort_order)
    grouped_df_corruption = grouped_df_corruption.sort_values(by='Sort Order').drop('Sort Order', axis=1)
    grouped_corruptions[corruption_folder] = grouped_df_corruption
    print(grouped_df_corruption)
    

In [ ]:

# a = ["clipart", "infograph", "painting", "quickdraw", "real", "sketch"]
list(grouped_corruptions[corruption_folders[5]]["Accuracy"])
list(grouped_corruptions[corruption_folders[5]]["Precision"])
list(grouped_corruptions[corruption_folders[5]]["Recall"])

## V2

In [4]:
import os
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the base folders
model_types = [
    "plumber_img_prompt", "plumber_img_text_proj_img_prompt_cls_LP",
    "plumber_img_proj", "plumber_img_prompt_cls_LP", 
    "plumber_text_proj_img_prompt", "plumber_img_proj_cls_LP", 
    "plumber_img_text_proj", "plumber_text_proj_img_prompt_cls_LP", 
    "plumber_img_proj_img_prompt", "plumber_img_text_proj_cls_LP", 
    "plumber_img_proj_img_prompt_cls_LP", "plumber_img_text_proj_img_prompt"
]
corruption_folders = [
    "brightness", "contrast", "defocus_blur", "elastic_transform", "fog",
    "frost", "gaussian_blur", "gaussian_noise", "glass_blur", "impulse_noise",
    "jpeg_compression", "motion_blur", "pixelate", "saturate", "shot_noise",
    "snow", "spatter", "speckle_noise", "zoom_blur"
]
corruption_folders = [f"cifar10-c_{folder}_4" for folder in corruption_folders]

corruption_folders = ["clipart", "infograph", "painting", "quickdraw", "real", "sketch"]
# corruption_folders = [f"{folder}" for folder in corruption_folders]

# corruption_folders = ["autumn", "dim", "grass", "outdoor", "rock", "water"]

model_types = ["plumber_img_prompt", "plumber_img_proj"]
corruption_folders = [""]
# corruption_folders = [corr+"/uq_AE" for corr in corruption_folders]

dataset = "Waterbirds"
save_data_name = "Waterbirds-ae-src"
model_name = "resnet18"

epoch = 99

# Directory where these folders are located
base_directory = f"logs/{dataset}/{model_name}"  # Replace with the actual path
remaining_dir = f"_clsEpoch_{epoch}_bs_128_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/AE-failure_detector/uq_AE_2_layer"

# # Directory where these folders are located
# base_directory = f"logs/{dataset}/failure_estimation/autumn/{model_name}"  # Replace with the actual path
# remaining_dir = f"_clsEpoch_99_bs_64_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/failure_detector"

def extract_metrics_from_json(file_path, model_type):
    # if model_type == "plumber_img_prompt" add update the model file_path to also include proj_features_metrics_single_svm and clip_features_metrics_single_svm
    if model_type == "plumber_img_prompt":
        file_paths = [file_path] + [file_path.replace("proj", "clip")]
    else:
        file_paths = [file_path]
    
    all_metrics = {}
    for file_path in file_paths:
        # Get the data type from the file path
        data_type = file_path.split("/")[-1].split("_metrics")[0]
        
        metrics = {}
        with open(file_path, 'r') as file:
            data = json.load(file)
        metrics['task_model_acc'] = accuracy_score(data["gt_labels"], data["task_pred"])
        metrics['estimated_acc'] = sum(data["correct_svm_pred"]) / len(data["correct_svm_pred"])
        metrics['estimation_gap'] = data["estimation_gap"]

        metrics['overall_svm_accuracy'] = data['class_report']['accuracy']

        metrics['failure_svm_accuracy'] = data["accu_failure_pred"]
        metrics['sucess_svm_accuracy'] = data["accu_success_pred"]

        all_metrics[data_type] = metrics

    return all_metrics

data_list = []
# Iterate over the base folders and corruption folders
for model_type in model_types:
    path = os.path.join(base_directory, model_type)
    for corruption_folder in corruption_folders:
        corruption_path = os.path.join(path, remaining_dir, f"{corruption_folder}")
        metrics_file = os.path.join(corruption_path, "proj_ae_metrics_single_svm.json")
        if os.path.exists(metrics_file):
            metrics_data = extract_metrics_from_json(metrics_file, model_type)
            print(metrics_data)
            for key, values in metrics_data.items():
                row = {"Model Type": model_type, 
                                "Corruption": corruption_folder,
                                "Feature Type": key,
                                **values}
                
                data_list.append(row)
        else:
            print("Metrics file not found for", metrics_file)

{'proj_ae': {'task_model_acc': 0.7506040731791509, 'estimated_acc': 0.5377977217811529, 'estimation_gap': 0.21280635139799797, 'overall_svm_accuracy': 0.591473938557128, 'failure_svm_accuracy': 0.6076124567474048, 'sucess_svm_accuracy': 0.5861117498275465}, 'clip_ae': {'task_model_acc': 0.7506040731791509, 'estimated_acc': 0.5032792544011045, 'estimation_gap': 0.24732481877804635, 'overall_svm_accuracy': 0.5662754573696928, 'failure_svm_accuracy': 0.6262975778546713, 'sucess_svm_accuracy': 0.5463324902276385}}
{'proj_ae': {'task_model_acc': 0.7506040731791509, 'estimated_acc': 0.5726613738350017, 'estimation_gap': 0.1779426993441492, 'overall_svm_accuracy': 0.612875388332758, 'failure_svm_accuracy': 0.5806228373702422, 'sucess_svm_accuracy': 0.6235916302598299}}


In [12]:
# Display the dataframe
df = pd.DataFrame(data_list)

# Save the dataframe
df.to_csv(f"{save_data_name}-{model_name}.csv", index=False)


# Group by 'Model Type' and 'Feature Type' and calculate mean for specific columns
grouped_df = df.groupby(['Model Type', 'Feature Type', 'Corruption']).agg({
    'task_model_acc': 'mean',
    'estimated_acc': 'mean',
    'estimation_gap': 'mean',
    'overall_svm_accuracy': 'mean',
    'failure_svm_accuracy': 'mean',
    'sucess_svm_accuracy': 'mean'
}).reset_index()

model_mapping = {
    "Image (Prompt)": "plumber_img_prompt",
    "Image (Project)": "plumber_img_proj",
    "Image (Prompt + Project)": "plumber_img_proj_img_prompt",
    "Image (Prompt) + Text (Cls Prompt)": "plumber_img_prompt_cls_LP",
    "Image (Prompt) + Text (Project)": "plumber_text_proj_img_prompt",
    "Image (Prompt) + Text (Prompt + Project)": "plumber_text_proj_img_prompt_cls_LP",
    "Image (Project) + Text (Cls Prompt)": "plumber_img_proj_cls_LP",
    "Image (Project) + Text (Project)": "plumber_img_text_proj",
    "Image (Project) + Text (Prompt + Project)": "plumber_img_text_proj_cls_LP",
    "Image (Prompt + Project) + Text (Cls Prompt)": "plumber_img_proj_img_prompt_cls_LP",
    "Image (Prompt + Project) + Text (Project)": "plumber_img_text_proj_img_prompt",
    "Image (Prompt + Project) + Text (Prompt + Project)": "plumber_img_text_proj_img_prompt_cls_LP"
}
# Reverse the dictionary for replacement (value to key)
reverse_mapping = {v: k for k, v in model_mapping.items()}

# Replace the 'Model Type' in the DataFrame using the reversed mapping
grouped_df['Model Type'] = grouped_df['Model Type'].map(reverse_mapping)

# Create a custom sort order based on the order of keys in model_mapping
sort_order = {k: i for i, k in enumerate(model_mapping.keys())}

# Add a temporary sorting column based on the custom order
grouped_df['Sort Order'] = grouped_df['Model Type'].map(sort_order)

# Sort the DataFrame by this custom order and drop the temporary column
grouped_df = grouped_df.sort_values(by='Sort Order').drop('Sort Order', axis=1)

# Display the sorted DataFrame
print(grouped_df)
grouped_df.to_csv(f"{save_data_name}-{model_name}-grouped.csv", index=False)

         Model Type Feature Type       Corruption  task_model_acc  \
8    Image (Prompt)      clip_ae   painting/uq_AE        0.312586   
15   Image (Prompt)      proj_ae  quickdraw/uq_AE        0.034512   
14   Image (Prompt)      proj_ae   painting/uq_AE        0.312586   
13   Image (Prompt)      proj_ae  infograph/uq_AE        0.111603   
12   Image (Prompt)      proj_ae    clipart/uq_AE        0.331485   
11   Image (Prompt)      clip_ae     sketch/uq_AE        0.200277   
10   Image (Prompt)      clip_ae       real/uq_AE        0.711727   
9    Image (Prompt)      clip_ae  quickdraw/uq_AE        0.034512   
17   Image (Prompt)      proj_ae     sketch/uq_AE        0.200277   
7    Image (Prompt)      clip_ae  infograph/uq_AE        0.111603   
6    Image (Prompt)      clip_ae    clipart/uq_AE        0.331485   
16   Image (Prompt)      proj_ae       real/uq_AE        0.711727   
5   Image (Project)      proj_ae     sketch/uq_AE        0.200277   
4   Image (Project)      proj_ae  

In [ ]:
# list(grouped_df["Accuracy"])
# list(grouped_df["Precision"])
# list(grouped_df["Recall"])

# Shift Detection

In [ ]:
import os
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the base folders
model_types = [
    "clip", "plumber_img_prompt", "plumber_img_text_proj_img_prompt_dataset_LP",
    "plumber_img_proj", "plumber_img_prompt_dataset_LP", 
    "plumber_text_proj_img_prompt", "plumber_img_proj_dataset_LP", 
    "plumber_img_text_proj", "plumber_text_proj_img_prompt_dataset_LP", 
    "plumber_img_proj_img_prompt", "plumber_img_text_proj_dataset_LP", 
    "plumber_img_proj_img_prompt_dataset_LP", "plumber_img_text_proj_img_prompt"
]

dataset = "domainnet"
domain = "real"
concept_set = "domainnet_domain_concepts"


# Directory where these folders are located
base_directory = f"logs/{dataset}/shift_detection/{domain}"  # Replace with the actual path

remaining_dir = f"_bs_256_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/dissect/{concept_set}/concept_similarity"

def extract_metrics_from_json(file_path, model_type):

    with open(file_path, 'r') as file:
        data = json.load(file)
       
    return data

data_list = []
# Iterate over the base folders and corruption folders
for model_type in model_types:
    path = os.path.join(base_directory, model_type, remaining_dir)
    metrics_file = os.path.join(path, "concept_similarity.json")
    if os.path.exists(metrics_file):
        metrics_data = extract_metrics_from_json(metrics_file, model_type)
        
        # add model type to the metrics data
        metrics_data["Model Type"] = model_type
        print(metrics_data)
            
        data_list.append(metrics_data)
    else:
        print("Metrics file not found for", metrics_file)
        data_list.append({"Model Type": model_type})

In [ ]:
# Display the dataframe
df = pd.DataFrame(data_list)

# # Save the dataframe
# df.to_csv(f"{save_data_name}-{model_name}.csv", index=False)


model_mapping = {
    "clip": "clip",
    "Image (Prompt)": "plumber_img_prompt",
    "Image (Project)": "plumber_img_proj",
    "Image (Prompt + Project)": "plumber_img_proj_img_prompt",
    "Image (Prompt) + Text (Dataset Prompt)": "plumber_img_prompt_dataset_LP",
    "Image (Prompt) + Text (Project)": "plumber_text_proj_img_prompt",
    "Image (Prompt) + Text (Dataset Prompt + Project)": "plumber_text_proj_img_prompt_dataset_LP",
    "Image (Project) + Text (Dataset Prompt)": "plumber_img_proj_dataset_LP",
    "Image (Project) + Text (Project)": "plumber_img_text_proj",
    "Image (Project) + Text (Dataset Prompt + Project)": "plumber_img_text_proj_dataset_LP",
    "Image (Prompt + Project) + Text (Dataset Prompt)": "plumber_img_proj_img_prompt_dataset_LP",
    "Image (Prompt + Project) + Text (Project)": "plumber_img_text_proj_img_prompt",
    "Image (Prompt + Project) + Text (Dataset Prompt + Project)": "plumber_img_text_proj_img_prompt_dataset_LP"
}
# Reverse the dictionary for replacement (value to key)
reverse_mapping = {v: k for k, v in model_mapping.items()}

# Replace the 'Model Type' in the DataFrame using the reversed mapping
df['Model Type'] = df['Model Type'].map(reverse_mapping)

# Create a custom sort order based on the order of keys in model_mapping
sort_order = {k: i for i, k in enumerate(model_mapping.keys())}

# Add a temporary sorting column based on the custom order
df['Sort Order'] = df['Model Type'].map(sort_order)

# Sort the DataFrame by this custom order and drop the temporary column
df = df.sort_values(by='Sort Order').drop('Sort Order', axis=1)

# Display the sorted DataFrame
print(df)
df.to_csv(f"SD:{dataset}-{domain}.csv", index=False)

# Zeroshot Abilities

In [ ]:
import os
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the base folders
model_types = [
    "clip", "plumber_img_prompt", "plumber_img_text_proj_img_prompt_dataset_LP",
    "plumber_img_proj", "plumber_dataset_LP", "plumber_text_proj",
    "plumber_text_proj_dataset_LP", "plumber_img_prompt_dataset_LP", 
    "plumber_text_proj_img_prompt", "plumber_img_proj_dataset_LP", 
    "plumber_img_text_proj", "plumber_text_proj_img_prompt_dataset_LP", 
    "plumber_img_proj_img_prompt", "plumber_img_text_proj_dataset_LP", 
    "plumber_img_proj_img_prompt_dataset_LP", "plumber_img_text_proj_img_prompt"
]
zsl_datasets=["cifar10", "cifar100", "gtsrb", "svhn", "dtd", "oxfordpets",  "food101", "eurosat", "sun397", "ucf101", "stanfordcars", "flowers102"]
zsl_datasets=["cifar10", "cifar100"]
dataset = "domainnet"
domain = "real"


# Directory where these folders are located
base_directory = f"logs/{dataset}/shift_detection/{domain}"  # Replace with the actual path

remaining_dir = f"_bs_256_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/logs"


data_list = []
# Iterate over the base folders and corruption folders
for model_type in model_types:
    path = os.path.join(base_directory, model_type, remaining_dir)
    zsl_results = {
        "Model Type": model_type,
    }
    for zsl_dataset in zsl_datasets:
        # Read the csv file and extract the test accuracy
        csv_file = os.path.join(path, f"{zsl_dataset}results.csv")
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            test_acc = df.iloc[-1]["Test Acc"]
        else:
            print("Metrics file not found for", csv_file)
            test_acc = None
        zsl_results[zsl_dataset] = test_acc
    
    data_list.append(zsl_results)

    

    # if os.path.exists(metrics_file):
    #     metrics_data = extract_metrics_from_json(metrics_file, model_type)
        
    #     # add model type to the metrics data
    #     metrics_data["Model Type"] = model_type
    #     print(metrics_data)
            
    #     data_list.append(metrics_data)
    # else:
    #     print("Metrics file not found for", metrics_file)
    #     data_list.append({"Model Type": model_type})

In [ ]:
# Display the dataframe
df = pd.DataFrame(data_list)

print(df)
# # Save the dataframe
# df.to_csv(f"{save_data_name}-{model_name}.csv", index=False)


model_mapping = {
    "clip": "clip",
    "Image (Prompt)": "plumber_img_prompt",
    "Image (Project)": "plumber_img_proj",
    "Image (Prompt + Project)": "plumber_img_proj_img_prompt",
    "Text (Dataset Prompt)": "plumber_dataset_LP",
    "Text (Project)": "plumber_text_proj",
    "Text (Dataset Prompt + Project)": "plumber_text_proj_dataset_LP",
    "Image (Prompt) + Text (Dataset Prompt)": "plumber_img_prompt_dataset_LP",
    "Image (Prompt) + Text (Project)": "plumber_text_proj_img_prompt",
    "Image (Prompt) + Text (Dataset Prompt + Project)": "plumber_text_proj_img_prompt_dataset_LP",
    "Image (Project) + Text (Dataset Prompt)": "plumber_img_proj_dataset_LP",
    "Image (Project) + Text (Project)": "plumber_img_text_proj",
    "Image (Project) + Text (Dataset Prompt + Project)": "plumber_img_text_proj_dataset_LP",
    "Image (Prompt + Project) + Text (Dataset Prompt)": "plumber_img_proj_img_prompt_dataset_LP",
    "Image (Prompt + Project) + Text (Project)": "plumber_img_text_proj_img_prompt",
    "Image (Prompt + Project) + Text (Dataset Prompt + Project)": "plumber_img_text_proj_img_prompt_dataset_LP"
}
# Reverse the dictionary for replacement (value to key)
reverse_mapping = {v: k for k, v in model_mapping.items()}

# Replace the 'Model Type' in the DataFrame using the reversed mapping
df['Model Type'] = df['Model Type'].map(reverse_mapping)

# Create a custom sort order based on the order of keys in model_mapping
sort_order = {k: i for i, k in enumerate(model_mapping.keys())}

# Add a temporary sorting column based on the custom order
df['Sort Order'] = df['Model Type'].map(sort_order)

# Sort the DataFrame by this custom order and drop the temporary column
df = df.sort_values(by='Sort Order').drop('Sort Order', axis=1)

# Display the sorted DataFrame
print(df)
df.to_csv(f"ZSL:{dataset}-{domain}.csv", index=False)

In [6]:
import clip

clip_model, clip_preprocess = clip.load("ViT-B/32", device="cpu")

print(clip_model)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [37]:
import torch
checkpoint = torch.load('logs/cifar10/SimpleCNN/masked_img_encoder/_clsEpoch_29_bs_256_lr_0.1_teT_2.0_sT_1.0_imgweight_1.0_txtweight_1.0_is_mlp_False/step_1/projector_weights_16.pth',
                        map_location=torch.device('cpu'))

In [40]:
a = torch.zeros(50,1,768)

b = checkpoint['masked_visual_encoder']["masks.10"]

print(torch.sum(a), torch.sum(b))
print(torch.sum(a-b))

torch.allclose(a[1,:,:], b.float()[1,:,:])


tensor(0.) tensor(82.6875, dtype=torch.float16)
tensor(-82.6609)


False

In [39]:
b

tensor([[[ 0.0138, -0.0089, -0.0053,  ..., -0.0090,  0.0002,  0.0078]],

        [[ 0.0140, -0.0045, -0.0007,  ...,  0.0036, -0.0054, -0.0057]],

        [[ 0.0133,  0.0040, -0.0057,  ..., -0.0080,  0.0598,  0.0060]],

        ...,

        [[ 0.0039,  0.0145, -0.0017,  ...,  0.0040,  0.0025, -0.0067]],

        [[ 0.0086,  0.0067,  0.0020,  ..., -0.0064,  0.0070,  0.0064]],

        [[ 0.0051, -0.0140, -0.0004,  ...,  0.0001,  0.0077,  0.0008]]],
       dtype=torch.float16)